In [ ]:
! pip install langchain
! pip install langchain-community
! pip install langchain-openai
! pip install duckduckgo-search
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00


In [ ]:
import os
import time
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from typing import List, Dict, Any

# Configurar la API key de OpenAI
os.environ["OPENAI_API_KEY"]

def log_separator(message=""):
    """Imprime un separador con un mensaje opcional"""
    width = 80
    if message:
        padding = (width - len(message) - 2) // 2
        print("\n" + "=" * padding + f" {message} " + "=" * padding)
    else:
        print("\n" + "=" * width)

def realizar_busqueda(query: str, num_results: int = 3) -> List[Dict[str, Any]]:
    """
    Realiza una búsqueda en DuckDuckGo y devuelve los resultados
    """
    log_separator("BÚSQUEDA")
    print(f"🔍 Realizando búsqueda en DuckDuckGo: '{query}'")
    print(f"🔍 Solicitando {num_results} resultados")

    start_time = time.time()
    search = DuckDuckGoSearchAPIWrapper()
    results = search.results(query, num_results)
    elapsed = time.time() - start_time

    print(f"✅ Búsqueda completada en {elapsed:.2f} segundos")
    print(f"✅ Se encontraron {len(results)} resultados:")

    for i, result in enumerate(results, 1):
        print(f"  [{i}] {result.get('title', 'Sin título')}")
        print(f"      URL: {result.get('link', 'Sin enlace')}")

    return results

def cargar_y_procesar_pagina(url: str, title: str, index: int) -> Dict[str, Any]:
    """
    Carga y procesa una página web completa, dividiéndola en chunks si es necesario
    """
    log_separator(f"PROCESANDO PÁGINA {index}")
    print(f"📄 Cargando página {index}: {title}")
    print(f"📄 URL: {url}")

    start_time = time.time()

    try:
        print(f"📥 Descargando contenido...")
        loader = WebBaseLoader(url)
        documents = loader.load()
        download_time = time.time() - start_time
        print(f"📥 Descarga completada en {download_time:.2f} segundos")

        if not documents:
            print(f"⚠️ AVISO: No se pudo extraer contenido de {url}")
            return {
                "title": title,
                "url": url,
                "success": False,
                "content": None,
                "chunks": [],
                "error": "No se pudo extraer contenido",
                "stats": {
                    "download_time": download_time,
                    "processing_time": 0,
                    "total_time": download_time
                }
            }

        content = documents[0].page_content
        content_chars = len(content)
        content_words = len(content.split())

        print(f"📊 ESTADÍSTICAS DEL DOCUMENTO:")
        print(f"  • Caracteres: {content_chars:,}")
        print(f"  • Palabras aproximadas: {content_words:,}")
        print(f"  • Tokens estimados: ~{int(content_chars/4):,} (estimación basada en 4 caracteres/token)")

        # Dividir el contenido en chunks manejables
        print(f"✂️ Dividiendo documento en chunks...")
        chunk_start = time.time()

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=4000,         # Tamaño aproximado de cada chunk
            chunk_overlap=200,       # Solapamiento entre chunks para mantener contexto
            separators=["\n\n", "\n", ". ", " ", ""],  # Prioridad de separadores
        )

        chunks = text_splitter.split_text(content)
        chunk_time = time.time() - chunk_start

        print(f"✂️ División completada en {chunk_time:.2f} segundos")
        print(f"✂️ Documento dividido en {len(chunks)} chunks")

        # Mostrar estadísticas de los chunks
        for i, chunk in enumerate(chunks, 1):
            chunk_chars = len(chunk)
            chunk_words = len(chunk.split())
            print(f"  • Chunk {i}: {chunk_chars:,} caracteres, ~{chunk_words:,} palabras, ~{int(chunk_chars/4):,} tokens")

        total_time = time.time() - start_time

        return {
            "title": title,
            "url": url,
            "success": True,
            "content": content,  # Contenido completo
            "chunks": chunks,    # Contenido dividido en chunks
            "error": None,
            "stats": {
                "download_time": download_time,
                "processing_time": chunk_time,
                "total_time": total_time,
                "total_chars": content_chars,
                "total_words": content_words,
                "num_chunks": len(chunks),
                "chunk_sizes": [len(chunk) for chunk in chunks]
            }
        }

    except Exception as e:
        elapsed = time.time() - start_time
        print(f"❌ ERROR: Error al procesar {url}: {str(e)}")
        return {
            "title": title,
            "url": url,
            "success": False,
            "content": None,
            "chunks": [],
            "error": str(e),
            "stats": {
                "download_time": elapsed,
                "processing_time": 0,
                "total_time": elapsed
            }
        }

def resumir_pagina_larga(pagina: Dict[str, Any], pregunta: str, index: int) -> str:
    """
    Procesa una página muy larga dividida en chunks y produce un resumen
    """
    if not pagina["success"] or not pagina["chunks"]:
        return f"No se pudo analizar la página: {pagina['error'] or 'Contenido no disponible'}"

    log_separator(f"RESUMIENDO PÁGINA {index}")
    print(f"📝 Resumiendo página {index}: {pagina['title']}")
    print(f"📝 Procesando {len(pagina['chunks'])} chunks")

    start_time = time.time()

    # Convertir los chunks a formato Document de LangChain
    chunks_docs = [
        Document(page_content=chunk, metadata={"source": pagina["url"], "title": pagina["title"]})
        for chunk in pagina["chunks"]
    ]

    # Inicializar el modelo
    print(f"🤖 Inicializando modelo GPT-3.5-turbo para resumir")
    llm = ChatOpenAI(temperature=0.3, model_name="gpt-3.5-turbo")

    # Prompt para resumir cada chunk
    print(f"📋 Configurando prompts de mapeo y reducción")
    map_prompt_template = """
    Tu tarea es extraer información relevante del siguiente fragmento de una página web
    que responda a esta pregunta del usuario: "{question}"

    Solo extrae información directamente relacionada con la pregunta. Sé conciso.

    FRAGMENTO DE PÁGINA WEB:
    {text}

    INFORMACIÓN RELEVANTE EXTRAÍDA (solo si está relacionada con la pregunta):
    """

    map_prompt = PromptTemplate(
        template=map_prompt_template,
        input_variables=["text", "question"]
    )

    # Prompt para combinar los resúmenes de los chunks
    combine_prompt_template = """
    Has recibido extractos de información de diferentes partes de una página web titulada "{title}".
    Usa estos extractos para crear un resumen coherente que responda a la pregunta del usuario.

    PREGUNTA DEL USUARIO: {question}

    EXTRACTOS DE LA PÁGINA WEB:
    {text}

    RESUMEN COHERENTE Y COMPLETO:
    """

    combine_prompt = PromptTemplate(
        template=combine_prompt_template,
        input_variables=["title", "text", "question"]
    )

    # Cargar la cadena de resumen
    print(f"⚙️ Creando cadena de resumen Map-Reduce")
    summary_chain = load_summarize_chain(
        llm,
        chain_type="map_reduce",
        map_prompt=map_prompt,
        combine_prompt=combine_prompt,
        verbose=False
    )

    try:
        # Ejecutar la cadena de resumen con los chunks
        print(f"🔄 Ejecutando cadena de resumen Map-Reduce...")
        print(f"🔄 Fase de Map: Procesando {len(chunks_docs)} chunks individualmente")
        summary_start = time.time()

        summary = summary_chain.invoke({
            "input_documents": chunks_docs,
            "question": pregunta,
            "title": pagina["title"]
        })

        summary_time = time.time() - summary_start
        print(f"✅ Resumen completado en {summary_time:.2f} segundos")

        # Extraer el resultado
        if isinstance(summary, dict) and "output_text" in summary:
            result = summary["output_text"]
        else:
            result = str(summary)

        result_chars = len(result)
        result_words = len(result.split())

        print(f"📊 ESTADÍSTICAS DEL RESUMEN:")
        print(f"  • Caracteres: {result_chars:,}")
        print(f"  • Palabras aproximadas: {result_words:,}")
        print(f"  • Tokens estimados: ~{int(result_chars/4):,}")
        print(f"  • Ratio de compresión: {result_chars/pagina['stats']['total_chars']*100:.1f}% del original")

        return result

    except Exception as e:
        elapsed = time.time() - start_time
        print(f"❌ ERROR: Error al resumir: {str(e)}")
        return f"Error al resumir la página: {str(e)}"

def analizar_resultados_completos(resultados_busqueda: List[Dict[str, Any]], pregunta: str) -> str:
    """
    Procesa los resultados de búsqueda, carga cada página web, y genera una respuesta
    """
    log_separator("ANÁLISIS DE RESULTADOS")
    print(f"🔍 Procesando {len(resultados_busqueda)} resultados de búsqueda")

    start_time = time.time()

    # Procesar cada página encontrada
    paginas_procesadas = []

    for i, result in enumerate(resultados_busqueda, 1):
        title = result.get("title", "Sin título")
        url = result.get("link", "Sin enlace")

        # Cargar y procesar la página
        pagina_procesada = cargar_y_procesar_pagina(url, title, i)

        if pagina_procesada["success"]:
            # Si la página es muy larga (tiene múltiples chunks), resumirla
            if len(pagina_procesada["chunks"]) > 1:
                resumen = resumir_pagina_larga(pagina_procesada, pregunta, i)
                pagina_procesada["resumen"] = resumen
            # Si la página es corta, usar el contenido directamente
            else:
                print(f"ℹ️ La página {i} tiene un solo chunk, no requiere resumen")
                pagina_procesada["resumen"] = pagina_procesada["content"]

        paginas_procesadas.append(pagina_procesada)

    # Preparar el contexto final para GPT
    log_separator("PREPARACIÓN DEL CONTEXTO FINAL")
    print(f"📋 Preparando contexto final para respuesta...")

    contexto_items = []
    total_contexto_chars = 0

    for i, pagina in enumerate(paginas_procesadas, 1):
        if pagina["success"]:
            contenido = pagina.get('resumen', pagina['content'])
            contexto_item = f"[FUENTE {i}] {pagina['title']}\n[URL] {pagina['url']}\n\n{contenido}"
            chars = len(contenido)
            total_contexto_chars += chars
            print(f"  • Fuente {i}: {pagina['title']} ({chars:,} caracteres)")
        else:
            contexto_item = f"[FUENTE {i}] {pagina['title']}\n[URL] {pagina['url']}\n\nNo se pudo obtener el contenido: {pagina['error']}"
            print(f"  • Fuente {i}: {pagina['title']} (NO DISPONIBLE - {pagina['error']})")

        contexto_items.append(contexto_item)

    # Unir el contexto final
    contexto_final = "\n\n---\n\n".join(contexto_items)
    contexto_final_chars = len(contexto_final)
    contexto_final_tokens = int(contexto_final_chars/4)

    print(f"📊 ESTADÍSTICAS DEL CONTEXTO FINAL:")
    print(f"  • Total caracteres: {contexto_final_chars:,}")
    print(f"  • Tokens estimados: ~{contexto_final_tokens:,}")

    if contexto_final_tokens > 15000:
        print(f"⚠️ ADVERTENCIA: El contexto final excede los 15,000 tokens aproximados.")
        print(f"⚠️ Se usará gpt-3.5-turbo-16k que puede manejar hasta ~16k tokens.")

    # Generar la respuesta final con GPT
    log_separator("GENERACIÓN DE RESPUESTA FINAL")
    print(f"🤖 Inicializando modelo para respuesta final...")

    template = """
    Responde a la siguiente pregunta basándote en el contenido de las páginas web proporcionadas.
    Para cada afirmación importante en tu respuesta, cita la fuente correspondiente usando [FUENTE X].

    PREGUNTA: {question}

    CONTENIDO DE LAS PÁGINAS WEB:
    {context}

    RESPUESTA DETALLADA (citando las fuentes):
    """

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=template
    )

    # Usar un modelo con contexto amplio para la respuesta final
    if contexto_final_tokens > 15000:
        print(f"🤖 Usando modelo gpt-3.5-turbo-16k para manejar el contexto amplio")
        llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo-16k")
    else:
        print(f"🤖 Usando modelo gpt-3.5-turbo estándar")
        llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

    chain = LLMChain(llm=llm, prompt=prompt)

    try:
        print(f"🔄 Enviando contexto a GPT para respuesta final...")
        resp_start = time.time()

        respuesta = chain.invoke({
            "context": contexto_final,
            "question": pregunta
        })

        resp_time = time.time() - resp_start
        print(f"✅ Respuesta generada en {resp_time:.2f} segundos")

        if isinstance(respuesta, dict) and "text" in respuesta:
            result = respuesta["text"]
        else:
            result = respuesta

        result_chars = len(result)
        result_words = len(result.split())

        print(f"📊 ESTADÍSTICAS DE LA RESPUESTA FINAL:")
        print(f"  • Caracteres: {result_chars:,}")
        print(f"  • Palabras aproximadas: {result_words:,}")

        return result

    except Exception as e:
        print(f"❌ ERROR: Error al generar respuesta final: {str(e)}")
        return f"Error al generar la respuesta: {str(e)}"

def buscar_y_responder(pregunta: str, num_resultados: int = 4) -> str:
    """
    Función principal que gestiona todo el proceso
    """
    log_separator("INICIO DE PROCESO")
    print(f"🔍 Procesando pregunta: {pregunta}")
    print(f"🔍 Número de resultados a procesar: {num_resultados}")

    global_start = time.time()

    # Realizar búsqueda
    resultados = realizar_busqueda(pregunta, num_resultados)

    # Analizar resultados y generar respuesta
    respuesta = analizar_resultados_completos(resultados, pregunta)

    total_time = time.time() - global_start

    log_separator("PROCESO COMPLETADO")
    print(f"⏱️ Tiempo total de ejecución: {total_time:.2f} segundos")

    return respuesta

# Ejemplo de uso
if __name__ == "__main__":
    pregunta_usuario = "¿Como programar y entrenar un mamba llm?"
    num_resultados = 4

    log_separator("CONFIGURACIÓN")
    print(f"❓ Pregunta: {pregunta_usuario}")
    print(f"🔢 Número de resultados: {num_resultados}")

    respuesta = buscar_y_responder(pregunta_usuario, num_resultados)

    log_separator("RESULTADO FINAL")
    print(respuesta)
    log_separator()


================================ CONFIGURACIÓN ================================
❓ Pregunta: ¿Como programar y entrenar un mamba llm?
🔢 Número de resultados: 4

============================== INICIO DE PROCESO ==============================
🔍 Procesando pregunta: ¿Como programar y entrenar un mamba llm?
🔍 Número de resultados a procesar: 4

=================================== BÚSQUEDA ===================================
🔍 Realizando búsqueda en DuckDuckGo: '¿Como programar y entrenar un mamba llm?'
🔍 Solicitando 4 resultados
✅ Búsqueda completada en 0.82 segundos
✅ Se encontraron 4 resultados:
  [1] Introducción a la arquitectura Mamba LLM: Un nuevo ... - DataCamp
      URL: https://www.datacamp.com/es/tutorial/introduction-to-the-mamba-llm-architecture
  [2] Entrenamiento de LLM con Datasets Guía Paso a Paso
      URL: https://decolornaranja.wordpress.com/2024/09/05/como-entrenar-un-modelo-de-lenguaje-llm-usando-un-dataset-paso-a-paso/
  [3] Una guía paso a paso para entrenar modelos 